In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder , OneHotEncoder
import pickle

In [2]:
data= pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocessing the dataset
### Dropping unnecessary columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1) ## axis=1 means we are dropping columns
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
## Encoding categorical variables
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
# Encoding categorical variables for Geography
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
geography_encoded = one_hot_encoder.fit_transform(data[['Geography']])
geography_encoded


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
one_hot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df = pd.DataFrame( geography_encoded.toarray() , columns=one_hot_encoder.get_feature_names_out(['Geography']))

In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## Concatenating the encoded geography columns with the original dataframe
data = pd.concat([data.drop('Geography',axis = 1), geo_encoded_df], axis=1)

In [10]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
## Split the data into features and target
X = data.drop('EstimatedSalary',axis = 1)
y = data['EstimatedSalary']

In [12]:
## Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
## Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
## SAving the scaler
with open('scaler_reg.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Regression Problem statement

In [15]:
import tensorflow

In [16]:
import tensorflow as tf

In [17]:
from tensorflow import keras


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 

In [20]:
## Building the model 
model = Sequential([
  Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
  Dense(32,activation='relu'),
  Dense(1) ## output Layer of regression (Deafault activation function is linear activation function )
])

In [27]:
model.compile(
  optimizer ='adam',
  loss = 'mean_absolute_error',
  metrics = ['mae']
)

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

In [30]:
## set up the Tensorboard
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [31]:
## Set up Early Stopping
## Lets say after some epochs we get the model at its peak performance and after that the variation from loss function is not that much 
## So instead of processing for all the epochs we can stop the training early
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [32]:
## Training the Model
history = model.fit(
  X_train,y_train,validation_data=(X_test,y_test),epochs=100,
  callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 4ms/step - loss: 71143.5703 - mae: 71143.5703 - val_loss: 67197.6719 - val_mae: 67197.6719
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 66095.7500 - mae: 66095.7500 - val_loss: 62283.9453 - val_mae: 62283.9453
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 59316.7930 - mae: 59316.7930 - val_loss: 54477.8359 - val_mae: 54477.8359
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 52620.9883 - mae: 52620.9883 - val_loss: 50924.0117 - val_mae: 50924.0117
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 51406.7109 - mae: 51406.7109 - val_loss: 50552.1406 - val_mae: 50552.1406
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 51196.4062 - mae: 51196.4062 - val_loss: 51035.4727 - val_mae: 51035.4727
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 51258.3984 - mae: 51258.3984 

In [33]:
%load_ext tensorboard

In [34]:
%tensorboard --logdir regressionlogs/fit

In [35]:
test_loss , test_mae = model.evaluate(X_test,y_test)
print(f"Test mae : {test_mae}")

63/63 [==============================] - 0s 3ms/step - loss: 50311.2148 - mae: 50311.2148
Test mae : 50311.21484375


In [36]:
model.save('regression_model.h5')

c:\Users\Admin\OneDrive\Desktop\ML projects\Bank_Binary_classification\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
